# MBA

In [532]:
df = df_clean.copy()
df = df.reset_index(drop=True)

In [533]:
df.head()

,Order_ID,Phone_new,Source,PaymentType,Region,FullSum,Quant,RowDiscount,RowSum,TN,TK
0,1303000526_TT,55575456-52485253545172,Онлайн-Резерв.,при получении,Центральный ф.о.,3498.0,1.0,0.0,2599.0,Мелкая бытовая техника,Приготовление пищи
1,1303000526_TT,55575456-52485253545172,Онлайн-Резерв.,при получении,Центральный ф.о.,3498.0,1.0,0.0,649.0,Аксессуары,Электропитание
2,1303000530_TT,55575050-50535349495672,Онлайн-Резерв.,при получении,Центральный ф.о.,54497.0,1.0,0.0,19999.0,Цифровая Техника,Мобильные средства связи
3,1303000530_TT,55575050-50535349495672,Онлайн-Резерв.,при получении,Центральный ф.о.,54497.0,1.0,0.0,19999.0,Цифровая Техника,Мобильные средства связи
4,1303000530_TT,55575050-50535349495672,Онлайн-Резерв.,при получении,Центральный ф.о.,54497.0,1.0,0.0,14499.0,Цифровая Техника,Мобильные средства связи


In [534]:
df['TN_TK'] = df['TN'] + '_' + df['TK']

In [535]:
df.head()

,Order_ID,Phone_new,Source,PaymentType,Region,FullSum,Quant,RowDiscount,RowSum,TN,TK,TN_TK
0,1303000526_TT,55575456-52485253545172,Онлайн-Резерв.,при получении,Центральный ф.о.,3498.0,1.0,0.0,2599.0,Мелкая бытовая техника,Приготовление пищи,Мелкая бытовая техника_Приготовление пищи
1,1303000526_TT,55575456-52485253545172,Онлайн-Резерв.,при получении,Центральный ф.о.,3498.0,1.0,0.0,649.0,Аксессуары,Электропитание,Аксессуары_Электропитание
2,1303000530_TT,55575050-50535349495672,Онлайн-Резерв.,при получении,Центральный ф.о.,54497.0,1.0,0.0,19999.0,Цифровая Техника,Мобильные средства связи,Цифровая Техника_Мобильные средства связи
3,1303000530_TT,55575050-50535349495672,Онлайн-Резерв.,при получении,Центральный ф.о.,54497.0,1.0,0.0,19999.0,Цифровая Техника,Мобильные средства связи,Цифровая Техника_Мобильные средства связи
4,1303000530_TT,55575050-50535349495672,Онлайн-Резерв.,при получении,Центральный ф.о.,54497.0,1.0,0.0,14499.0,Цифровая Техника,Мобильные средства связи,Цифровая Техника_Мобильные средства связи


In [536]:
df['TN_TK'].value_counts()

TN_TK
Крупная бытовая техника_Встраиваемая техника    1893
Сделай сам_Сантехника                           1739
Мелкая бытовая техника_Приготовление пищи       1675
Сделай сам_Инструменты                           777
Товары для дома_Хозтовары                        747
                                                ... 
ТВ-Аудио_DVD/Blu-Ray плееры и компоненты           1
Крупная бытовая техника_Сушильные машины           1
ТВ-Аудио_Hi-Fi компоненты                          1
Детские товары_Для мам                             1
Спорт и активный отдых_Рыбалка                     1
Name: count, Length: 95, dtype: int64

In [537]:
# транзакционный DF
df_tr = df.copy()

# Добавляем колонку unit_price
df_tr['unit_price'] = df_tr['RowSum'] / df_tr['Quant']

# Группируем по товару (TN) и считаем среднюю unit_price
avg_price_by_TN = (
    df_tr
    .groupby('TN')['unit_price']
    .mean()
    .rename('avg_price')
    .reset_index()
)

In [538]:
price_map = avg_price_by_TN.set_index('TN')['avg_price']

df['avg_price'] = df['TN'].map(price_map)

#Проверяем
df.head()
df_with_price = df.copy()

In [539]:
# create a dataframe to record customer information
data_mba = pd.DataFrame()
data_mba['id'] = df.drop_duplicates(subset = 'Phone_new')['Phone_new']
data_mba = data_mba.reset_index(drop=True)

In [540]:
data_mba.head()

,id
0,55575456-52485253545172
1,55575050-50535349495672
2,55574954-49565455575577
3,55574954-50515050575778
4,55575050-50485353485575


In [541]:
#column groupings
group_TN_TK = df.groupby(['Phone_new', 'TN_TK']).sum()

In [542]:
for i in df.TN_TK.unique():
    data_mba[i] = i
    data_mba = data_mba.merge(group_TN_TK['Quant'], left_on = ['id', i], right_on = ['Phone_new', 'TN_TK'], how = 'left')
    data_mba = data_mba.drop(columns = [i])
    data_mba = data_mba.rename(columns = {'Quant': i})
    data_mba = data_mba.fillna(0)

In [543]:
data_mba.head()

,id,Мелкая бытовая техника_Приготовление пищи,Аксессуары_Электропитание,Цифровая Техника_Мобильные средства связи,Цифровая Техника_Портативная электроника,Аксессуары_Аксессуары для комп./ноутбуков/принтеров,Климат_Вентиляторы бытовые,Климат_Увлажнители,Аксессуары_Аксессуары для бытовой техники,Мелкая бытовая техника_Красота и здоровье,Сделай сам_Сантехника,Аксессуары_Аксессуары для смартфонов и планшетов,Детские товары_Детская комната,Аксессуары_Носители информации,Цифровая Техника_Фотоаппараты,Мелкая бытовая техника_СВЧ,ТВ-Аудио_Телевизоры LCD,Крупная бытовая техника_Встраиваемая техника,Крупная бытовая техника_Стиральные машины,Товары для дома_Хозтовары,Спорт и активный отдых_Тяжелая атлетика,Товары для дома_Товары для кухни,Спорт и активный отдых_Зимние виды спорта,Мелкая бытовая техника_Уход за домом,Мелкая бытовая техника_Швейная техника,Аксессуары_Аксессуары ТВ-Аудио,Аксессуары_Сетевое оборудование,Сделай сам_Силовая и садовая техника,Сделай сам_Инструменты,Аксессуары_Наушники,Детские товары_Игрушки,Крупная бытовая техника_Плиты,Крупная бытовая техника_Холодильное оборудование,Цифровая Техника_Периферийные устройства,Спорт и активный отдых_Велосипеды и аксессуары,Товары для дома_Текстиль,Спорт и активный отдых_Аксессуары для спортивных товаров,ТВ-Аудио_Аудио-Видео,Спорт и активный отдых_Фитнес,Детские товары_Кормление,Аксессуары_Картриджи и бумага,Мебель_Аксессуары для мебели,Климат_Кондиционеры,Цифровая Техника_Планшеты,Спорт и активный отдых_Тренажеры,Крупная бытовая техника_Посудомоечные машины,Мебель_Кухня и Столовая,Цифровая Техника_Компьютеры настольные,Цифровая Техника_Компьютеры портативные,Цифровая Техника_Средства связи,Мебель_Гостиная,Мебель_Спальня,Мебель_Офис и Кабинет,Аксессуары_Аксессуары для фото/видео,Товары для дома_Декор интерьера,ТВ-Аудио_Автоэлектроника,Мебель_Прихожая,Спорт и активный отдых_Командные виды спорта,Детские товары_Прогулки и путешествие,"Игры, софт и развлечения_Игры для консолей",Спорт и активный отдых_Туризм,Климат_Воздухонагреватели,Спорт и активный отдых_Активный отдых,ТВ-Аудио_Музыкальные инструменты,ТВ-Аудио_Аксессуары авто,"Игры, софт и развлечения_Игровые консоли",Детские товары_Купание и уход,Товары для дома_Свет,Цифровая Техника_Видеокамеры,Детские товары_Аксессуары детство,Сделай сам_Камины,Мебель_Мягкая мебель,Товары для дома_Сервировка стола,Климат_Очистители воздуха,Климат_Водонагреватели,"Игры, софт и развлечения_Аксессуары для игровых приставок",Аксессуары_Освещение,"Игры, софт и развлечения_Антивирусные программы",ТВ-Аудио_Автомагнитолы,Аксессуары_Музыкальные инструменты,ТВ-Аудио_Домашние театры,"Игры, софт и развлечения_DVD с записью",Сделай сам_Товары для автолюбителей,Мебель_Ванная,Спорт и активный отдых_Единоборства,Автотовары_Автопринадлежности,"Игры, софт и развлечения_Операционные системы",ТВ-Аудио_Аудиомагнитолы,Аксессуары_Умный дом,ТВ-Аудио_Музыкальные центры,Спорт и активный отдых_Индивидуальные виды спорта,ТВ-Аудио_DVD/Blu-Ray плееры и компоненты,Крупная бытовая техника_Сушильные машины,ТВ-Аудио_Hi-Fi компоненты,Детские товары_Для мам,Спорт и активный отдых_Рыбалка
0,55575456-52485253545172,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,55575050-50535349495672,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,55574954-49565455575577,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0

In [544]:
data_mba.shape

(8772, 96)

In [545]:
df_clusters = pd.read_excel('labels_for_7_klasters_EM.xlsx')

In [546]:
df2 = df_clusters

In [547]:
df2.head()

,id,Мелкая бытовая техника,Аксессуары,Цифровая Техника,Климат,Сделай сам,Детские товары,ТВ-Аудио,Крупная бытовая техника,Товары для дома,Спорт и активный отдых,Мебель,"Игры, софт и развлечения",Автотовары,Онлайн-Резерв.,Колл-центр,Сайт,Терминал,при получении,предоплата онлайн,Центральный ф.о.,Северо-Западный ф.о.,Сибирский ф.о.,Южный ф.о.,Уральский ф.о.,Приволжский ф.о.,Дальневосточный ф.о.,СУММА СКИДКИ,КОЛ-ВО ЧЕКОВ,СРЕД.КОЛ.ТОВ.вЧеке,СРЕДНИЙ ЧЕК,Labels
0,55575456-52485253545172,1.160155,1.219993,-0.368224,-0.174947,-0.395253,-0.261512,-0.251942,-0.444735,-0.33083,-0.315504,-0.277897,-0.05498,-0.025649,-0.124414,-0.395109,-0.095794,-0.040229,-0.229124,-0.307744,0.306473,-0.464922,-0.188374,-0.173004,-0.151419,-0.296013,-0.026014,-0.223170,-0.184614,-0.474434,-0.633739,2
1,55575050-50535349495672,-0.486567,-0.544559,4.474704,-0.174947,-0.395253,-0.261512,-0.251942,-0.444735,-0.33083,-0.315504,-0.277897,-0.05498,-0.025649,0.542751,-0.395109,-0.095794,-0.040229,0.483190,-0.307744,0.931638,-0.464922,-0.188374,-0.173004,-0.151419,-0.296013,-0.026014,-0.223170,-0.184614,0.500058,2.109759,2
2,55574954-49565455575577,1.160155,-0.544559,2.053240,-0.174947,-0.395253,-0.261512,-0.251942,-0.444735,-0.33083,-0.315504,-0.277897,-0.05498,-0.025649,-0.124414,-0.395109,-0.095794,-0.040229,-0.229124,-0.307744,-0.943856,1.157351,-0.188374,-0.173004,-0.151419,-0.296013,-0.026014,-0.109777,-0.184614,-0.474434,-0.729562,2
3,55574954-50515050575778,-0.486567,1.219993,-0.368224,-0.174947,-0.395253,-0.261512,-0.251942,-0.444735,-0.33083,-0.315504,1.877781,-0.05498,-0.025649,1.209917,-0.395109,-0.095794,-0.040229,1.195503,-0.307744,-0.943856,2.779624,-0.188374,-0.173004,-0.151419,-0.296013,-0.026014,-0.223170,4.388703,-0.474434,-0.681222,1
4,55575050-50485353485575,-0.486567,2.984546,-0.368224,-0.174947,-0.395253,-0.261512,-0.251942,-0.444735,-0.33083,-0.315504,-0.277897,-0.05498,-0.025649,-0.124414,-0.395109,-0.095794,-0.040229,-0.229124,-0.307744,0.306473,-0.464922,-0.188374,-0.173004,-0.151419,-0.296013,-0.026014,-0.223170,-0.184614,-0.474434,-0.780418,2


In [548]:
#добавление столбца с номером кластера
data_mba['Cluster'] = None

In [549]:
#из таблицы сегментации заполняем столбец с номерами кластеров по ключу id
keys = ['id']
data_mba = data_mba.assign(Cluster=data_mba[keys].join(df2.drop_duplicates(keys).set_index(keys).Labels, on=keys).Labels)

In [550]:
data_mba.head()

,id,Мелкая бытовая техника_Приготовление пищи,Аксессуары_Электропитание,Цифровая Техника_Мобильные средства связи,Цифровая Техника_Портативная электроника,Аксессуары_Аксессуары для комп./ноутбуков/принтеров,Климат_Вентиляторы бытовые,Климат_Увлажнители,Аксессуары_Аксессуары для бытовой техники,Мелкая бытовая техника_Красота и здоровье,Сделай сам_Сантехника,Аксессуары_Аксессуары для смартфонов и планшетов,Детские товары_Детская комната,Аксессуары_Носители информации,Цифровая Техника_Фотоаппараты,Мелкая бытовая техника_СВЧ,ТВ-Аудио_Телевизоры LCD,Крупная бытовая техника_Встраиваемая техника,Крупная бытовая техника_Стиральные машины,Товары для дома_Хозтовары,Спорт и активный отдых_Тяжелая атлетика,Товары для дома_Товары для кухни,Спорт и активный отдых_Зимние виды спорта,Мелкая бытовая техника_Уход за домом,Мелкая бытовая техника_Швейная техника,Аксессуары_Аксессуары ТВ-Аудио,Аксессуары_Сетевое оборудование,Сделай сам_Силовая и садовая техника,Сделай сам_Инструменты,Аксессуары_Наушники,Детские товары_Игрушки,Крупная бытовая техника_Плиты,Крупная бытовая техника_Холодильное оборудование,Цифровая Техника_Периферийные устройства,Спорт и активный отдых_Велосипеды и аксессуары,Товары для дома_Текстиль,Спорт и активный отдых_Аксессуары для спортивных товаров,ТВ-Аудио_Аудио-Видео,Спорт и активный отдых_Фитнес,Детские товары_Кормление,Аксессуары_Картриджи и бумага,Мебель_Аксессуары для мебели,Климат_Кондиционеры,Цифровая Техника_Планшеты,Спорт и активный отдых_Тренажеры,Крупная бытовая техника_Посудомоечные машины,Мебель_Кухня и Столовая,Цифровая Техника_Компьютеры настольные,Цифровая Техника_Компьютеры портативные,Цифровая Техника_Средства связи,Мебель_Гостиная,Мебель_Спальня,Мебель_Офис и Кабинет,Аксессуары_Аксессуары для фото/видео,Товары для дома_Декор интерьера,ТВ-Аудио_Автоэлектроника,Мебель_Прихожая,Спорт и активный отдых_Командные виды спорта,Детские товары_Прогулки и путешествие,"Игры, софт и развлечения_Игры для консолей",Спорт и активный отдых_Туризм,Климат_Воздухонагреватели,Спорт и активный отдых_Активный отдых,ТВ-Аудио_Музыкальные инструменты,ТВ-Аудио_Аксессуары авто,"Игры, софт и развлечения_Игровые консоли",Детские товары_Купание и уход,Товары для дома_Свет,Цифровая Техника_Видеокамеры,Детские товары_Аксессуары детство,Сделай сам_Камины,Мебель_Мягкая мебель,Товары для дома_Сервировка стола,Климат_Очистители воздуха,Климат_Водонагреватели,"Игры, софт и развлечения_Аксессуары для игровых приставок",Аксессуары_Освещение,"Игры, софт и развлечения_Антивирусные программы",ТВ-Аудио_Автомагнитолы,Аксессуары_Музыкальные инструменты,ТВ-Аудио_Домашние театры,"Игры, софт и развлечения_DVD с записью",Сделай сам_Товары для автолюбителей,Мебель_Ванная,Спорт и активный отдых_Единоборства,Автотовары_Автопринадлежности,"Игры, софт и развлечения_Операционные системы",ТВ-Аудио_Аудиомагнитолы,Аксессуары_Умный дом,ТВ-Аудио_Музыкальные центры,Спорт и активный отдых_Индивидуальные виды спорта,ТВ-Аудио_DVD/Blu-Ray плееры и компоненты,Крупная бытовая техника_Сушильные машины,ТВ-Аудио_Hi-Fi компоненты,Детские товары_Для мам,Спорт и активный отдых_Рыбалка,Cluster
0,55575456-52485253545172,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0
1,55575050-50535349495672,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0
2,55574954-49565455575577,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0

In [551]:
data_mba.to_csv('Data_for_MBA_all_clusters_true.csv')

**MBA analysis for cluster 0**

In [552]:
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules

In [553]:
df = pd.read_csv('Data_for_MBA_all_clusters_true.csv')
df.set_index('id', inplace = True)
df.drop(['Unnamed: 0'], axis = 1, inplace = True)

In [554]:
df.head()

,Мелкая бытовая техника_Приготовление пищи,Аксессуары_Электропитание,Цифровая Техника_Мобильные средства связи,Цифровая Техника_Портативная электроника,Аксессуары_Аксессуары для комп./ноутбуков/принтеров,Климат_Вентиляторы бытовые,Климат_Увлажнители,Аксессуары_Аксессуары для бытовой техники,Мелкая бытовая техника_Красота и здоровье,Сделай сам_Сантехника,Аксессуары_Аксессуары для смартфонов и планшетов,Детские товары_Детская комната,Аксессуары_Носители информации,Цифровая Техника_Фотоаппараты,Мелкая бытовая техника_СВЧ,ТВ-Аудио_Телевизоры LCD,Крупная бытовая техника_Встраиваемая техника,Крупная бытовая техника_Стиральные машины,Товары для дома_Хозтовары,Спорт и активный отдых_Тяжелая атлетика,Товары для дома_Товары для кухни,Спорт и активный отдых_Зимние виды спорта,Мелкая бытовая техника_Уход за домом,Мелкая бытовая техника_Швейная техника,Аксессуары_Аксессуары ТВ-Аудио,Аксессуары_Сетевое оборудование,Сделай сам_Силовая и садовая техника,Сделай сам_Инструменты,Аксессуары_Наушники,Детские товары_Игрушки,Крупная бытовая техника_Плиты,Крупная бытовая техника_Холодильное оборудование,Цифровая Техника_Периферийные устройства,Спорт и активный отдых_Велосипеды и аксессуары,Товары для дома_Текстиль,Спорт и активный отдых_Аксессуары для спортивных товаров,ТВ-Аудио_Аудио-Видео,Спорт и активный отдых_Фитнес,Детские товары_Кормление,Аксессуары_Картриджи и бумага,Мебель_Аксессуары для мебели,Климат_Кондиционеры,Цифровая Техника_Планшеты,Спорт и активный отдых_Тренажеры,Крупная бытовая техника_Посудомоечные машины,Мебель_Кухня и Столовая,Цифровая Техника_Компьютеры настольные,Цифровая Техника_Компьютеры портативные,Цифровая Техника_Средства связи,Мебель_Гостиная,Мебель_Спальня,Мебель_Офис и Кабинет,Аксессуары_Аксессуары для фото/видео,Товары для дома_Декор интерьера,ТВ-Аудио_Автоэлектроника,Мебель_Прихожая,Спорт и активный отдых_Командные виды спорта,Детские товары_Прогулки и путешествие,"Игры, софт и развлечения_Игры для консолей",Спорт и активный отдых_Туризм,Климат_Воздухонагреватели,Спорт и активный отдых_Активный отдых,ТВ-Аудио_Музыкальные инструменты,ТВ-Аудио_Аксессуары авто,"Игры, софт и развлечения_Игровые консоли",Детские товары_Купание и уход,Товары для дома_Свет,Цифровая Техника_Видеокамеры,Детские товары_Аксессуары детство,Сделай сам_Камины,Мебель_Мягкая мебель,Товары для дома_Сервировка стола,Климат_Очистители воздуха,Климат_Водонагреватели,"Игры, софт и развлечения_Аксессуары для игровых приставок",Аксессуары_Освещение,"Игры, софт и развлечения_Антивирусные программы",ТВ-Аудио_Автомагнитолы,Аксессуары_Музыкальные инструменты,ТВ-Аудио_Домашние театры,"Игры, софт и развлечения_DVD с записью",Сделай сам_Товары для автолюбителей,Мебель_Ванная,Спорт и активный отдых_Единоборства,Автотовары_Автопринадлежности,"Игры, софт и развлечения_Операционные системы",ТВ-Аудио_Аудиомагнитолы,Аксессуары_Умный дом,ТВ-Аудио_Музыкальные центры,Спорт и активный отдых_Индивидуальные виды спорта,ТВ-Аудио_DVD/Blu-Ray плееры и компоненты,Крупная бытовая техника_Сушильные машины,ТВ-Аудио_Hi-Fi компоненты,Детские товары_Для мам,Спорт и активный отдых_Рыбалка,Cluster
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
55575456-52485253545172,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0
55575050-50535349495672,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0

In [555]:
#данные по кластеру 0
df_0 = df.loc[df['Cluster'] == 0]

In [556]:
df_0.shape

(537, 96)

In [557]:
MarkBas_0 = df_0.drop(columns='Cluster', axis=1)

In [558]:
MarkBas_0.head()

,Мелкая бытовая техника_Приготовление пищи,Аксессуары_Электропитание,Цифровая Техника_Мобильные средства связи,Цифровая Техника_Портативная электроника,Аксессуары_Аксессуары для комп./ноутбуков/принтеров,Климат_Вентиляторы бытовые,Климат_Увлажнители,Аксессуары_Аксессуары для бытовой техники,Мелкая бытовая техника_Красота и здоровье,Сделай сам_Сантехника,Аксессуары_Аксессуары для смартфонов и планшетов,Детские товары_Детская комната,Аксессуары_Носители информации,Цифровая Техника_Фотоаппараты,Мелкая бытовая техника_СВЧ,ТВ-Аудио_Телевизоры LCD,Крупная бытовая техника_Встраиваемая техника,Крупная бытовая техника_Стиральные машины,Товары для дома_Хозтовары,Спорт и активный отдых_Тяжелая атлетика,Товары для дома_Товары для кухни,Спорт и активный отдых_Зимние виды спорта,Мелкая бытовая техника_Уход за домом,Мелкая бытовая техника_Швейная техника,Аксессуары_Аксессуары ТВ-Аудио,Аксессуары_Сетевое оборудование,Сделай сам_Силовая и садовая техника,Сделай сам_Инструменты,Аксессуары_Наушники,Детские товары_Игрушки,Крупная бытовая техника_Плиты,Крупная бытовая техника_Холодильное оборудование,Цифровая Техника_Периферийные устройства,Спорт и активный отдых_Велосипеды и аксессуары,Товары для дома_Текстиль,Спорт и активный отдых_Аксессуары для спортивных товаров,ТВ-Аудио_Аудио-Видео,Спорт и активный отдых_Фитнес,Детские товары_Кормление,Аксессуары_Картриджи и бумага,Мебель_Аксессуары для мебели,Климат_Кондиционеры,Цифровая Техника_Планшеты,Спорт и активный отдых_Тренажеры,Крупная бытовая техника_Посудомоечные машины,Мебель_Кухня и Столовая,Цифровая Техника_Компьютеры настольные,Цифровая Техника_Компьютеры портативные,Цифровая Техника_Средства связи,Мебель_Гостиная,Мебель_Спальня,Мебель_Офис и Кабинет,Аксессуары_Аксессуары для фото/видео,Товары для дома_Декор интерьера,ТВ-Аудио_Автоэлектроника,Мебель_Прихожая,Спорт и активный отдых_Командные виды спорта,Детские товары_Прогулки и путешествие,"Игры, софт и развлечения_Игры для консолей",Спорт и активный отдых_Туризм,Климат_Воздухонагреватели,Спорт и активный отдых_Активный отдых,ТВ-Аудио_Музыкальные инструменты,ТВ-Аудио_Аксессуары авто,"Игры, софт и развлечения_Игровые консоли",Детские товары_Купание и уход,Товары для дома_Свет,Цифровая Техника_Видеокамеры,Детские товары_Аксессуары детство,Сделай сам_Камины,Мебель_Мягкая мебель,Товары для дома_Сервировка стола,Климат_Очистители воздуха,Климат_Водонагреватели,"Игры, софт и развлечения_Аксессуары для игровых приставок",Аксессуары_Освещение,"Игры, софт и развлечения_Антивирусные программы",ТВ-Аудио_Автомагнитолы,Аксессуары_Музыкальные инструменты,ТВ-Аудио_Домашние театры,"Игры, софт и развлечения_DVD с записью",Сделай сам_Товары для автолюбителей,Мебель_Ванная,Спорт и активный отдых_Единоборства,Автотовары_Автопринадлежности,"Игры, софт и развлечения_Операционные системы",ТВ-Аудио_Аудиомагнитолы,Аксессуары_Умный дом,ТВ-Аудио_Музыкальные центры,Спорт и активный отдых_Индивидуальные виды спорта,ТВ-Аудио_DVD/Blu-Ray плееры и компоненты,Крупная бытовая техника_Сушильные машины,ТВ-Аудио_Hi-Fi компоненты,Детские товары_Для мам,Спорт и активный отдых_Рыбалка
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
55575053-51484851505176,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
55575149-50515157505374,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
55575656

In [559]:
#Create function to hot encode the values
def encode_values(x):
    if x <= 0:
        return 0
    if x >= 1:
        return 1

#Apply function to data
basket_encoded_0 = MarkBas_0.applymap(encode_values)

basket_encoded_0.head()

/var/folders/m_/q7bdmmy93879kj5hcdgd5gsc0000gq/T/ipykernel_48164/1760409210.py:9: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  basket_encoded_0 = MarkBas_0.applymap(encode_values)


,Мелкая бытовая техника_Приготовление пищи,Аксессуары_Электропитание,Цифровая Техника_Мобильные средства связи,Цифровая Техника_Портативная электроника,Аксессуары_Аксессуары для комп./ноутбуков/принтеров,Климат_Вентиляторы бытовые,Климат_Увлажнители,Аксессуары_Аксессуары для бытовой техники,Мелкая бытовая техника_Красота и здоровье,Сделай сам_Сантехника,Аксессуары_Аксессуары для смартфонов и планшетов,Детские товары_Детская комната,Аксессуары_Носители информации,Цифровая Техника_Фотоаппараты,Мелкая бытовая техника_СВЧ,ТВ-Аудио_Телевизоры LCD,Крупная бытовая техника_Встраиваемая техника,Крупная бытовая техника_Стиральные машины,Товары для дома_Хозтовары,Спорт и активный отдых_Тяжелая атлетика,Товары для дома_Товары для кухни,Спорт и активный отдых_Зимние виды спорта,Мелкая бытовая техника_Уход за домом,Мелкая бытовая техника_Швейная техника,Аксессуары_Аксессуары ТВ-Аудио,Аксессуары_Сетевое оборудование,Сделай сам_Силовая и садовая техника,Сделай сам_Инструменты,Аксессуары_Наушники,Детские товары_Игрушки,Крупная бытовая техника_Плиты,Крупная бытовая техника_Холодильное оборудование,Цифровая Техника_Периферийные устройства,Спорт и активный отдых_Велосипеды и аксессуары,Товары для дома_Текстиль,Спорт и активный отдых_Аксессуары для спортивных товаров,ТВ-Аудио_Аудио-Видео,Спорт и активный отдых_Фитнес,Детские товары_Кормление,Аксессуары_Картриджи и бумага,Мебель_Аксессуары для мебели,Климат_Кондиционеры,Цифровая Техника_Планшеты,Спорт и активный отдых_Тренажеры,Крупная бытовая техника_Посудомоечные машины,Мебель_Кухня и Столовая,Цифровая Техника_Компьютеры настольные,Цифровая Техника_Компьютеры портативные,Цифровая Техника_Средства связи,Мебель_Гостиная,Мебель_Спальня,Мебель_Офис и Кабинет,Аксессуары_Аксессуары для фото/видео,Товары для дома_Декор интерьера,ТВ-Аудио_Автоэлектроника,Мебель_Прихожая,Спорт и активный отдых_Командные виды спорта,Детские товары_Прогулки и путешествие,"Игры, софт и развлечения_Игры для консолей",Спорт и активный отдых_Туризм,Климат_Воздухонагреватели,Спорт и активный отдых_Активный отдых,ТВ-Аудио_Музыкальные инструменты,ТВ-Аудио_Аксессуары авто,"Игры, софт и развлечения_Игровые консоли",Детские товары_Купание и уход,Товары для дома_Свет,Цифровая Техника_Видеокамеры,Детские товары_Аксессуары детство,Сделай сам_Камины,Мебель_Мягкая мебель,Товары для дома_Сервировка стола,Климат_Очистители воздуха,Климат_Водонагреватели,"Игры, софт и развлечения_Аксессуары для игровых приставок",Аксессуары_Освещение,"Игры, софт и развлечения_Антивирусные программы",ТВ-Аудио_Автомагнитолы,Аксессуары_Музыкальные инструменты,ТВ-Аудио_Домашние театры,"Игры, софт и развлечения_DVD с записью",Сделай сам_Товары для автолюбителей,Мебель_Ванная,Спорт и активный отдых_Единоборства,Автотовары_Автопринадлежности,"Игры, софт и развлечения_Операционные системы",ТВ-Аудио_Аудиомагнитолы,Аксессуары_Умный дом,ТВ-Аудио_Музыкальные центры,Спорт и активный отдых_Индивидуальные виды спорта,ТВ-Аудио_DVD/Blu-Ray плееры и компоненты,Крупная бытовая техника_Сушильные машины,ТВ-Аудио_Hi-Fi компоненты,Детские товары_Для мам,Спорт и активный отдых_Рыбалка
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
55575053-51484851505176,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
55575149-50515157505374,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
55575656-48564951485276,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
55574853-535449565753745,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0

In [560]:
#filter for only invoices with 2 or more items
basket_filtered_0 = basket_encoded_0[(basket_encoded_0 > 0).sum(axis=1) >= 2]
basket_filtered_0.head()

,Мелкая бытовая техника_Приготовление пищи,Аксессуары_Электропитание,Цифровая Техника_Мобильные средства связи,Цифровая Техника_Портативная электроника,Аксессуары_Аксессуары для комп./ноутбуков/принтеров,Климат_Вентиляторы бытовые,Климат_Увлажнители,Аксессуары_Аксессуары для бытовой техники,Мелкая бытовая техника_Красота и здоровье,Сделай сам_Сантехника,Аксессуары_Аксессуары для смартфонов и планшетов,Детские товары_Детская комната,Аксессуары_Носители информации,Цифровая Техника_Фотоаппараты,Мелкая бытовая техника_СВЧ,ТВ-Аудио_Телевизоры LCD,Крупная бытовая техника_Встраиваемая техника,Крупная бытовая техника_Стиральные машины,Товары для дома_Хозтовары,Спорт и активный отдых_Тяжелая атлетика,Товары для дома_Товары для кухни,Спорт и активный отдых_Зимние виды спорта,Мелкая бытовая техника_Уход за домом,Мелкая бытовая техника_Швейная техника,Аксессуары_Аксессуары ТВ-Аудио,Аксессуары_Сетевое оборудование,Сделай сам_Силовая и садовая техника,Сделай сам_Инструменты,Аксессуары_Наушники,Детские товары_Игрушки,Крупная бытовая техника_Плиты,Крупная бытовая техника_Холодильное оборудование,Цифровая Техника_Периферийные устройства,Спорт и активный отдых_Велосипеды и аксессуары,Товары для дома_Текстиль,Спорт и активный отдых_Аксессуары для спортивных товаров,ТВ-Аудио_Аудио-Видео,Спорт и активный отдых_Фитнес,Детские товары_Кормление,Аксессуары_Картриджи и бумага,Мебель_Аксессуары для мебели,Климат_Кондиционеры,Цифровая Техника_Планшеты,Спорт и активный отдых_Тренажеры,Крупная бытовая техника_Посудомоечные машины,Мебель_Кухня и Столовая,Цифровая Техника_Компьютеры настольные,Цифровая Техника_Компьютеры портативные,Цифровая Техника_Средства связи,Мебель_Гостиная,Мебель_Спальня,Мебель_Офис и Кабинет,Аксессуары_Аксессуары для фото/видео,Товары для дома_Декор интерьера,ТВ-Аудио_Автоэлектроника,Мебель_Прихожая,Спорт и активный отдых_Командные виды спорта,Детские товары_Прогулки и путешествие,"Игры, софт и развлечения_Игры для консолей",Спорт и активный отдых_Туризм,Климат_Воздухонагреватели,Спорт и активный отдых_Активный отдых,ТВ-Аудио_Музыкальные инструменты,ТВ-Аудио_Аксессуары авто,"Игры, софт и развлечения_Игровые консоли",Детские товары_Купание и уход,Товары для дома_Свет,Цифровая Техника_Видеокамеры,Детские товары_Аксессуары детство,Сделай сам_Камины,Мебель_Мягкая мебель,Товары для дома_Сервировка стола,Климат_Очистители воздуха,Климат_Водонагреватели,"Игры, софт и развлечения_Аксессуары для игровых приставок",Аксессуары_Освещение,"Игры, софт и развлечения_Антивирусные программы",ТВ-Аудио_Автомагнитолы,Аксессуары_Музыкальные инструменты,ТВ-Аудио_Домашние театры,"Игры, софт и развлечения_DVD с записью",Сделай сам_Товары для автолюбителей,Мебель_Ванная,Спорт и активный отдых_Единоборства,Автотовары_Автопринадлежности,"Игры, софт и развлечения_Операционные системы",ТВ-Аудио_Аудиомагнитолы,Аксессуары_Умный дом,ТВ-Аудио_Музыкальные центры,Спорт и активный отдых_Индивидуальные виды спорта,ТВ-Аудио_DVD/Blu-Ray плееры и компоненты,Крупная бытовая техника_Сушильные машины,ТВ-Аудио_Hi-Fi компоненты,Детские товары_Для мам,Спорт и активный отдых_Рыбалка
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
55575053-51484851505176,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
55575149-50515157505374,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
55575656-48564951485276,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
55574853-535449565753745,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0

In [561]:
basket_filtered_0.shape

(439, 95)

In [562]:
basket_filtered_0.to_csv('mba_1.csv')

In [563]:
#Generate the frequent itemsets
frequent_itemsets_0 = apriori(basket_filtered_0, min_support=0.03, use_colnames=True).sort_values("support",ascending=False)
frequent_itemsets_0.head(10)

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/mlxtend/frequent_patterns/fpcommon.py:161: DeprecationWarning: DataFrames with non-bool types result in worse computationalperformance and their support might be discontinued in the future.Please use a DataFrame with bool type
  warnings.warn(


,support,itemsets
0,0.148064,(Мелкая бытовая техника_Приготовление пищи)
13,0.129841,(Крупная бытовая техника_Встраиваемая техника)
24,0.116173,(Крупная бытовая техника_Холодильное оборудова...
12,0.107062,(ТВ-Аудио_Телевизоры LCD)
14,0.107062,(Крупная бытовая техника_Стиральные машины)
15,0.091116,(Товары для дома_Хозтовары)
8,0.079727,(Сделай сам_Сантехника)
17,0.079727,(Товары для дома_Товары для кухни)
22,0.075171,(Детские товары_Игрушки)
28,0.072893,(Мебель_Аксессуары для мебели)


In [564]:
#Apply association rules
assoc_rules_0 = association_rules(frequent_itemsets_0, metric="lift", min_threshold=0.2).sort_values("lift",ascending=False).reset_index(drop=True)
assoc_rules_0.head()

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,representativity,leverage,conviction,zhangs_metric,jaccard,certainty,kulczynski
0,(ТВ-Аудио_Телевизоры LCD),(Аксессуары_Аксессуары ТВ-Аудио),0.107062,0.052392,0.036446,0.340426,6.497687,1.0,0.030837,1.436696,0.947545,0.296296,0.303959,0.518039
1,(Аксессуары_Аксессуары ТВ-Аудио),(ТВ-Аудио_Телевизоры LCD),0.052392,0.107062,0.036446,0.695652,6.497687,1.0,0.030837,2.933941,0.892879,0.296296,0.659161,0.518039
2,(Крупная бытовая техника_Холодильное оборудова...,(Крупная бытовая техника_Стиральные машины),0.116173,0.107062,0.052392,0.450980,4.212349,1.0,0.039954,1.626424,0.862842,0.306667,0.385154,0.470171
3,(Крупная бытовая техника_Стиральные машины),(Крупная бытовая техника_Холодильное оборудова...,0.107062,0.116173,0.052392,0.489362,4.212349,1.0,0.039954,1.730828,0.854037,0.306667,0.422242,0.470171
4,(Мелкая бытовая техника_Приготовление пищи),(Товары для дома_Товары для кухни),0.148064,0.079727,0.036446,0.246154,3.087473,1.0,0.024642,1.220771,0.793616,0.190476,0.180845,0.351648


In [565]:
price_map = (
    df_with_price[['TN','avg_price']]
    .drop_duplicates(subset='TN')
    .set_index('TN')['avg_price']
)

assoc_rules_0['consequent_item'] = assoc_rules_0['consequents'].apply(lambda s: next(iter(s)))
assoc_rules_0['consequent_TN']   = assoc_rules_0['consequent_item'].apply(lambda x: x.split('_',1)[0])

assoc_rules_0['avg_price_consequent'] = assoc_rules_0['consequent_TN'].map(price_map)

# Считаем score
assoc_rules_0['score'] = (
    assoc_rules_0['confidence']
  * assoc_rules_0['lift']
  * assoc_rules_0['avg_price_consequent']
)

# Сортируем
assoc_rules_0 = assoc_rules_0.sort_values('score', ascending=False)
assoc_rules_0[[
    'antecedents','consequents',
    'confidence','lift',
    'avg_price_consequent','score'
]].head(10)

,antecedents,consequents,confidence,lift,avg_price_consequent,score
1,(Аксессуары_Аксессуары ТВ-Аудио),(ТВ-Аудио_Телевизоры LCD),0.695652,6.497687,17472.296758,78977.058234
3,(Крупная бытовая техника_Стиральные машины),(Крупная бытовая техника_Холодильное оборудова...,0.489362,4.212349,18947.654681,39057.978449
2,(Крупная бытовая техника_Холодильное оборудова...,(Крупная бытовая техника_Стиральные машины),0.450980,4.212349,18947.654681,35994.607590
7,(Крупная бытовая техника_Холодильное оборудова...,(Крупная бытовая техника_Встраиваемая техника),0.372549,2.869281,18947.654681,20254.054555
6,(Крупная бытовая техника_Встраиваемая техника),(Крупная бытовая техника_Холодильное оборудова...,0.333333,2.869281,18947.654681,18122.048812
9,(Крупная бытовая техника_Стиральные машины),(Крупная бытовая техника_Встраиваемая техника),0.340426,2.621874,18947.654681,16911.782132
8,(Крупная бытовая техника_Встраиваемая техника),(Крупная бытовая техника_Стиральные машины),0.280702,2.621874,18947.654681,13944.802811
5,(Товары для дома_Товары для кухни),(Мелкая бытовая техника_Приготовление пищи),0.457143,3.087473,4329.630674,6110.910062
0,(ТВ-Аудио_Телевизоры LCD),(Аксессуары_Аксессуары ТВ-Аудио),0.340426,6.497687,1254.350720,2774.597031
4,(Мелкая бытовая техника_Приготовление пищи),(Товары для дома_Товары для кухни),0.246154,3.087473,1283.735692,975.630445


In [566]:
assoc_rules_0['antecedent_items'] = assoc_rules_0['antecedents'].apply(lambda s: list(s))
assoc_rules_0['consequent_item']  = assoc_rules_0['consequents'].apply(lambda s: next(iter(s)))

#Собираем какие товары уже купил каждый клиент
customer_items = df_with_price.groupby('Phone_new')['TN_TK'].apply(set)

recs = []
for _, rule in assoc_rules_0.iterrows():
    A = set(rule['antecedent_items'])
    C = rule['consequent_item']
    for phone, items in customer_items.items():
        if A.issubset(items) and C not in items:
            recs.append({
                'Phone_new':     phone,
                'antecedents':   ", ".join(rule['antecedent_items']),
                'recommend_consequents':     C,
                'confidence':    rule['confidence'],
                'lift':          rule['lift'],
                'avg_price':     rule['avg_price_consequent'],
                'score':         rule['score']
            })

recommend_df_0 = pd.DataFrame(recs).drop_duplicates()

recommend_df_0.head()

,Phone_new,antecedents,recommend_consequents,confidence,lift,avg_price,score
0,55574848-48555649515078,Аксессуары_Аксессуары ТВ-Аудио,ТВ-Аудио_Телевизоры LCD,0.695652,6.497687,17472.296758,78977.058234
1,55574850-535149515556785,Аксессуары_Аксессуары ТВ-Аудио,ТВ-Аудио_Телевизоры LCD,0.695652,6.497687,17472.296758,78977.058234
2,55574851-49505650565272,Аксессуары_Аксессуары ТВ-Аудио,ТВ-Аудио_Телевизоры LCD,0.695652,6.497687,17472.296758,78977.058234
3,55574851-49505651565279,Аксессуары_Аксессуары ТВ-Аудио,ТВ-Аудио_Телевизоры LCD,0.695652,6.497687,17472.296758,78977.058234
4,55574851-50535454565673,Аксессуары_Аксессуары ТВ-Аудио,ТВ-Аудио_Телевизоры LCD,0.695652,6.497687,17472.296758,78977.058234


**MBA analysis for cluster 1**

In [567]:
df = pd.read_csv('Data_for_MBA_all_clusters_true.csv')
df.set_index('id', inplace = True)
df.drop(['Unnamed: 0'], axis = 1, inplace = True)

In [568]:
df_1 = df.loc[df['Cluster'] == 1]

In [569]:
MarkBas_1 = df_1.drop(columns='Cluster', axis=1)

In [570]:
#Create function to hot encode the values
def encode_values(x):
    if x <= 0:
        return 0
    if x >= 1:
        return 1

#Apply function to data
basket_encoded_1 = MarkBas_1.applymap(encode_values)


/var/folders/m_/q7bdmmy93879kj5hcdgd5gsc0000gq/T/ipykernel_48164/901735650.py:9: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  basket_encoded_1 = MarkBas_1.applymap(encode_values)


In [571]:
basket_filtered_1 = basket_encoded_1[(basket_encoded_1 > 0).sum(axis=1) >= 2]

In [572]:
basket_filtered_1.shape

(872, 95)

In [573]:
basket_filtered_1.to_csv('mba_2.csv')

In [574]:
#Generate the frequent itemsets
frequent_itemsets_1 = apriori(basket_filtered_1, min_support=0.03, use_colnames=True).sort_values("support",ascending=False)
frequent_itemsets_1.head(10)

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/mlxtend/frequent_patterns/fpcommon.py:161: DeprecationWarning: DataFrames with non-bool types result in worse computationalperformance and their support might be discontinued in the future.Please use a DataFrame with bool type
  warnings.warn(


,support,itemsets
6,0.201835,(Товары для дома_Хозтовары)
0,0.188073,(Мелкая бытовая техника_Приготовление пищи)
7,0.188073,(Товары для дома_Товары для кухни)
12,0.162844,(Мебель_Аксессуары для мебели)
16,0.126147,(Мебель_Прихожая)
14,0.112385,(Мебель_Спальня)
10,0.112385,(Сделай сам_Инструменты)
8,0.111239,(Мелкая бытовая техника_Уход за домом)
9,0.106651,(Сделай сам_Силовая и садовая техника)
15,0.104358,(Мебель_Офис и Кабинет)


In [575]:
#Apply association rules
assoc_rules_1 = association_rules(frequent_itemsets_1, metric="lift", min_threshold=0.2).sort_values("lift",ascending=False).reset_index(drop=True)
assoc_rules_1

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,representativity,leverage,conviction,zhangs_metric,jaccard,certainty,kulczynski
0,(Сделай сам_Инструменты),(Сделай сам_Силовая и садовая техника),0.112385,0.106651,0.044725,0.397959,3.731402,1.0,0.032739,1.483867,0.824687,0.256579,0.326085,0.408657
1,(Сделай сам_Силовая и садовая техника),(Сделай сам_Инструменты),0.106651,0.112385,0.044725,0.419355,3.731402,1.0,0.032739,1.528670,0.819394,0.256579,0.345836,0.408657
2,(Мебель_Аксессуары для мебели),(Мебель_Прихожая),0.162844,0.126147,0.058486,0.359155,2.847119,1.0,0.037944,1.363595,0.774966,0.253731,0.266644,0.411396
3,(Мебель_Прихожая),(Мебель_Аксессуары для мебели),0.126147,0.162844,0.058486,0.463636,2.847119,1.0,0.037944,1.560799,0.742422,0.253731,0.359303,0.411396
4,(Мебель_Спальня),(Мебель_Прихожая),0.112385,0.126147,0.040138,0.357143,2.831169,1.0,0.025961,1.359327,0.728682,0.202312,0.264342,0.337662
5,(Мебель_Прихожая),(Мебель_Спальня),0.126147,0.112385,0.040138,0.318182,2.831169,1.0,0.025961,1.301835,0.740157,0.202312,0.231853,0.337662
6,(Товары для дома_Хозтовары),(Мелкая бытовая техника_Уход за домом),0.201835,0.111239,0.060780,0.301136,2.707123,1.0,0.038328,1.271724,0.790067,0.240909,0.213666,0.423764
7,(Мелкая бытовая техника_Уход за домом),(Товары для дома_Хозтовары),0.111239,0.201835,0.060780,0.546392,2.707123,1.0,0.038328,1.759591,0.709531,0.240909,0.431686,0.423764
8,(Мелкая бытовая техника_Приготовление пищи),(Товары для дома_Товары для кухни),0.188073,0.188073,0.071101,0.378049,2.010113,1.0,0.035729,1.305451,0.618917,0.233083,0.233981,0.378049
9,(Товары для дома_Товары для кухни),(Мелкая бытовая техника_Приготовление пищи),0.188073,0.188073,0.071101,0.378049,2.010113,1.0,0.035729,1.305451,0.618917,0.233083,0.233981,0.378049


In [576]:
price_map = (
    df_with_price[['TN','avg_price']]
    .drop_duplicates(subset='TN')
    .set_index('TN')['avg_price']
)


assoc_rules_1['consequent_item'] = assoc_rules_1['consequents'].apply(lambda s: next(iter(s)))
assoc_rules_1['consequent_TN']   = assoc_rules_1['consequent_item'].apply(lambda x: x.split('_',1)[0])


assoc_rules_1['avg_price_consequent'] = assoc_rules_1['consequent_TN'].map(price_map)

# Считаем score
assoc_rules_1['score'] = (
    assoc_rules_1['confidence']
  * assoc_rules_1['lift']
  * assoc_rules_1['avg_price_consequent']
)

# Сортируем
assoc_rules_1 = assoc_rules_1.sort_values('score', ascending=False)
assoc_rules_1[[
    'antecedents','consequents',
    'confidence','lift',
    'avg_price_consequent','score'
]].head(10)

,antecedents,consequents,confidence,lift,avg_price_consequent,score
1,(Сделай сам_Силовая и садовая техника),(Сделай сам_Инструменты),0.419355,3.731402,3328.979810,5209.126299
0,(Сделай сам_Инструменты),(Сделай сам_Силовая и садовая техника),0.397959,3.731402,3328.979810,4943.354549
3,(Мебель_Прихожая),(Мебель_Аксессуары для мебели),0.463636,2.847119,3100.533576,4092.790938
6,(Товары для дома_Хозтовары),(Мелкая бытовая техника_Уход за домом),0.301136,2.707123,4329.630674,3529.571679
9,(Товары для дома_Товары для кухни),(Мелкая бытовая техника_Приготовление пищи),0.378049,2.010113,4329.630674,3290.176314
2,(Мебель_Аксессуары для мебели),(Мебель_Прихожая),0.359155,2.847119,3100.533576,3170.471853
4,(Мебель_Спальня),(Мебель_Прихожая),0.357143,2.831169,3100.533576,3135.047865
5,(Мебель_Прихожая),(Мебель_Спальня),0.318182,2.831169,3100.533576,2793.042643
7,(Мелкая бытовая техника_Уход за домом),(Товары для дома_Хозтовары),0.546392,2.707123,1283.735692,1898.837080
8,(Мелкая бытовая техника_Приготовление пищи),(Товары для дома_Товары для кухни),0.378049,2.010113,1283.735692,975.537427


In [577]:
assoc_rules_1['antecedent_items'] = assoc_rules_1['antecedents'].apply(lambda s: list(s))
assoc_rules_1['consequent_item']  = assoc_rules_1['consequents'].apply(lambda s: next(iter(s)))

#Собираем какие товары уже купил каждый клиент
customer_items = df_with_price.groupby('Phone_new')['TN_TK'].apply(set)

recs = []
for _, rule in assoc_rules_1.iterrows():
    A = set(rule['antecedent_items'])
    C = rule['consequent_item']
    for phone, items in customer_items.items():
        if A.issubset(items) and C not in items:
            recs.append({
                'Phone_new':     phone,
                'antecedents':   ", ".join(rule['antecedent_items']),
                'recommend_consequents':     C,
                'confidence':    rule['confidence'],
                'lift':          rule['lift'],
                'avg_price':     rule['avg_price_consequent'],
                'score':         rule['score']
            })

recommend_df_1 = pd.DataFrame(recs).drop_duplicates()

recommend_df_1.head()

,Phone_new,antecedents,recommend_consequents,confidence,lift,avg_price,score
0,55574848-505656575752705,Сделай сам_Силовая и садовая техника,Сделай сам_Инструменты,0.419355,3.731402,3328.97981,5209.126299
1,55574848-57485550485670,Сделай сам_Силовая и садовая техника,Сделай сам_Инструменты,0.419355,3.731402,3328.97981,5209.126299
2,55574850-535453525248755,Сделай сам_Силовая и садовая техника,Сделай сам_Инструменты,0.419355,3.731402,3328.97981,5209.126299
3,55574851-484851525650755,Сделай сам_Силовая и садовая техника,Сделай сам_Инструменты,0.419355,3.731402,3328.97981,5209.126299
4,55574851-48485248544977,Сделай сам_Силовая и садовая техника,Сделай сам_Инструменты,0.419355,3.731402,3328.97981,5209.126299


In [578]:
assoc_rules_1.to_excel('Associacions_for_cluster_1.xlsx')

**Анализ MBA для кластера 2**

In [579]:
df = pd.read_csv('Data_for_MBA_all_clusters_true.csv')
df.set_index('id', inplace = True)
df.drop(['Unnamed: 0'], axis = 1, inplace = True)

In [580]:
#данные по кластеру 2
df_2 = df.loc[df['Cluster'] == 2]

In [581]:
MarkBas_2 = df_2.drop(columns='Cluster', axis=1)

In [582]:
#Create function to hot encode the values
def encode_values(x):
    if x <= 0:
        return 0
    if x >= 1:
        return 1

#Apply function to data
basket_encoded_2 = MarkBas_2.applymap(encode_values)

/var/folders/m_/q7bdmmy93879kj5hcdgd5gsc0000gq/T/ipykernel_48164/418734060.py:9: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  basket_encoded_2 = MarkBas_2.applymap(encode_values)


In [583]:
#filter for only invoices with 2 or more items
basket_filtered_2 = basket_encoded_2[(basket_encoded_2 > 0).sum(axis=1) >= 2]
#basket_filtered_1.head()

In [584]:
basket_filtered_2.shape

(1395, 95)

In [585]:
basket_filtered_2.to_csv('mba_3.csv')

In [586]:
#Generate the frequent itemsets
frequent_itemsets_2 = apriori(basket_filtered_2, min_support=0.03, use_colnames=True).sort_values("support",ascending=False)
frequent_itemsets_2.head()

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/mlxtend/frequent_patterns/fpcommon.py:161: DeprecationWarning: DataFrames with non-bool types result in worse computationalperformance and their support might be discontinued in the future.Please use a DataFrame with bool type
  warnings.warn(


,support,itemsets
2,0.207168,(Цифровая Техника_Мобильные средства связи)
7,0.197849,(Аксессуары_Аксессуары для смартфонов и планше...
8,0.191398,(Аксессуары_Носители информации)
4,0.132616,(Аксессуары_Аксессуары для комп./ноутбуков/при...
5,0.132616,(Аксессуары_Аксессуары для бытовой техники)


In [587]:
#Apply association rules
assoc_rules_2 = association_rules(frequent_itemsets_2, metric="lift", min_threshold=0.2).sort_values("lift",ascending=False).reset_index(drop=True)
assoc_rules_2.head()

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,representativity,leverage,conviction,zhangs_metric,jaccard,certainty,kulczynski
0,(ТВ-Аудио_Телевизоры LCD),(Аксессуары_Аксессуары ТВ-Аудио),0.111828,0.131900,0.094624,0.846154,6.415134,1.0,0.079874,5.642652,0.950400,0.634615,0.822778,0.781773
1,(Аксессуары_Аксессуары ТВ-Аудио),(ТВ-Аудио_Телевизоры LCD),0.131900,0.111828,0.094624,0.717391,6.415134,1.0,0.079874,3.142763,0.972374,0.634615,0.681809,0.781773
2,(Аксессуары_Картриджи и бумага),(Цифровая Техника_Периферийные устройства),0.067384,0.131183,0.055914,0.829787,6.325427,1.0,0.047074,5.104301,0.902738,0.391960,0.804087,0.628008
3,(Цифровая Техника_Периферийные устройства),(Аксессуары_Картриджи и бумага),0.131183,0.067384,0.055914,0.426230,6.325427,1.0,0.047074,1.625417,0.969028,0.391960,0.384773,0.628008
4,(Крупная бытовая техника_Стиральные машины),(Аксессуары_Аксессуары для бытовой техники),0.046595,0.132616,0.035842,0.769231,5.800416,1.0,0.029663,3.758662,0.868045,0.250000,0.733948,0.519751


In [588]:
price_map = (
    df_with_price[['TN','avg_price']]
    .drop_duplicates(subset='TN')
    .set_index('TN')['avg_price']
)


assoc_rules_2['consequent_item'] = assoc_rules_2['consequents'].apply(lambda s: next(iter(s)))
assoc_rules_2['consequent_TN']   = assoc_rules_2['consequent_item'].apply(lambda x: x.split('_',1)[0])


assoc_rules_2['avg_price_consequent'] = assoc_rules_2['consequent_TN'].map(price_map)

# Считаем score
assoc_rules_2['score'] = (
    assoc_rules_2['confidence']
  * assoc_rules_2['lift']
  * assoc_rules_2['avg_price_consequent']
)

# Сортируем
assoc_rules_2 = assoc_rules_2.sort_values('score', ascending=False)
assoc_rules_2[[
    'antecedents','consequents',
    'confidence','lift',
    'avg_price_consequent','score'
]].head(10)

,antecedents,consequents,confidence,lift,avg_price_consequent,score
1,(Аксессуары_Аксессуары ТВ-Аудио),(ТВ-Аудио_Телевизоры LCD),0.717391,6.415134,17472.296758,80410.326031
2,(Аксессуары_Картриджи и бумага),(Цифровая Техника_Периферийные устройства),0.829787,6.325427,11276.214820,59186.131803
5,(Аксессуары_Аксессуары для бытовой техники),(Крупная бытовая техника_Стиральные машины),0.270270,5.800416,18947.654681,29703.858268
13,(Аксессуары_Аксессуары для смартфонов и планше...,(Цифровая Техника_Мобильные средства связи),0.626812,3.025613,11276.214820,21385.221253
7,(Аксессуары_Носители информации),(ТВ-Аудио_Автоэлектроника),0.202247,4.408357,17472.296758,15577.911345
11,(Аксессуары_Аксессуары для смартфонов и планше...,(Цифровая Техника_Планшеты),0.239130,3.369565,11276.214820,9085.985949
15,(Аксессуары_Аксессуары для комп./ноутбуков/при...,(Цифровая Техника_Периферийные устройства),0.313514,2.389898,11276.214820,8448.877028
0,(ТВ-Аудио_Телевизоры LCD),(Аксессуары_Аксессуары ТВ-Аудио),0.846154,6.415134,1254.350720,6808.854187
4,(Крупная бытовая техника_Стиральные машины),(Аксессуары_Аксессуары для бытовой техники),0.769231,5.800416,1254.350720,5596.735182
9,(Аксессуары_Аксессуары для бытовой техники),(Мелкая бытовая техника_Уход за домом),0.281081,4.262045,4329.630674,5186.811469


In [589]:
assoc_rules_2['antecedent_items'] = assoc_rules_2['antecedents'].apply(lambda s: list(s))
assoc_rules_2['consequent_item']  = assoc_rules_2['consequents'].apply(lambda s: next(iter(s)))

#Собираем какие товары уже купил каждый клиент
customer_items = df_with_price.groupby('Phone_new')['TN_TK'].apply(set)

recs = []
for _, rule in assoc_rules_2.iterrows():
    A = set(rule['antecedent_items'])
    C = rule['consequent_item']
    for phone, items in customer_items.items():
        if A.issubset(items) and C not in items:
            recs.append({
                'Phone_new':     phone,
                'antecedents':   ", ".join(rule['antecedent_items']),
                'recommend_consequents':     C,
                'confidence':    rule['confidence'],
                'lift':          rule['lift'],
                'avg_price':     rule['avg_price_consequent'],
                'score':         rule['score']
            })

recommend_df_2 = pd.DataFrame(recs).drop_duplicates()

recommend_df_2.head()

,Phone_new,antecedents,recommend_consequents,confidence,lift,avg_price,score
0,55574848-48555649515078,Аксессуары_Аксессуары ТВ-Аудио,ТВ-Аудио_Телевизоры LCD,0.717391,6.415134,17472.296758,80410.326031
1,55574850-535149515556785,Аксессуары_Аксессуары ТВ-Аудио,ТВ-Аудио_Телевизоры LCD,0.717391,6.415134,17472.296758,80410.326031
2,55574851-49505650565272,Аксессуары_Аксессуары ТВ-Аудио,ТВ-Аудио_Телевизоры LCD,0.717391,6.415134,17472.296758,80410.326031
3,55574851-49505651565279,Аксессуары_Аксессуары ТВ-Аудио,ТВ-Аудио_Телевизоры LCD,0.717391,6.415134,17472.296758,80410.326031
4,55574851-50535454565673,Аксессуары_Аксессуары ТВ-Аудио,ТВ-Аудио_Телевизоры LCD,0.717391,6.415134,17472.296758,80410.326031


**Анализ MBA для кластера 3**

In [590]:
df = pd.read_csv('Data_for_MBA_all_clusters_true.csv')
df.set_index('id', inplace = True)
df.drop(['Unnamed: 0'], axis = 1, inplace = True)

In [591]:
df_3 = df.loc[df['Cluster'] == 3]

In [592]:
MarkBas_3 = df_3.drop(columns='Cluster', axis=1)

In [593]:
#Create function to hot encode the values
def encode_values(x):
    if x <= 0:
        return 0
    if x >= 1:
        return 1

#Apply function to data
basket_encoded_3 = MarkBas_3.applymap(encode_values)

/var/folders/m_/q7bdmmy93879kj5hcdgd5gsc0000gq/T/ipykernel_48164/2971195922.py:9: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  basket_encoded_3 = MarkBas_3.applymap(encode_values)


In [594]:
#filter for only invoices with 2 or more items
basket_filtered_3 = basket_encoded_3[(basket_encoded_3 > 0).sum(axis=1) >= 2]

In [595]:
basket_filtered_3.shape

(439, 95)

In [596]:
basket_filtered_3.to_csv('mba_4.csv')

In [597]:
#Generate the frequent itemsets
frequent_itemsets_3 = apriori(basket_filtered_3, min_support=0.03, use_colnames=True).sort_values("support",ascending=False)
frequent_itemsets_3.head()

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/mlxtend/frequent_patterns/fpcommon.py:161: DeprecationWarning: DataFrames with non-bool types result in worse computationalperformance and their support might be discontinued in the future.Please use a DataFrame with bool type
  warnings.warn(


,support,itemsets
4,0.182232,(Аксессуары_Аксессуары для бытовой техники)
5,0.148064,(Аксессуары_Аксессуары для смартфонов и планше...
2,0.143508,(Цифровая Техника_Мобильные средства связи)
16,0.136674,(Цифровая Техника_Периферийные устройства)
8,0.120729,(Крупная бытовая техника_Стиральные машины)


In [598]:
#Apply association rules
assoc_rules_3 = association_rules(frequent_itemsets_3, metric="lift", min_threshold=0.2).sort_values("lift",ascending=False).reset_index(drop=True)
assoc_rules_3.head()

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,representativity,leverage,conviction,zhangs_metric,jaccard,certainty,kulczynski
0,(Аксессуары_Аксессуары ТВ-Аудио),(ТВ-Аудио_Телевизоры LCD),0.104784,0.107062,0.088838,0.847826,7.919056,1.0,0.077620,5.867882,0.975990,0.722222,0.829581,0.838807
1,(ТВ-Аудио_Телевизоры LCD),(Аксессуары_Аксессуары ТВ-Аудио),0.107062,0.104784,0.088838,0.829787,7.919056,1.0,0.077620,5.259396,0.978480,0.722222,0.809864,0.838807
2,(Аксессуары_Картриджи и бумага),(Цифровая Техника_Периферийные устройства),0.063781,0.136674,0.061503,0.964286,7.055357,1.0,0.052786,24.173121,0.916734,0.442623,0.958632,0.707143
3,(Цифровая Техника_Периферийные устройства),(Аксессуары_Картриджи и бумага),0.136674,0.063781,0.061503,0.450000,7.055357,1.0,0.052786,1.702216,0.994137,0.442623,0.412530,0.707143
4,(Аксессуары_Аксессуары для смартфонов и планше...,(Цифровая Техника_Мобильные средства связи),0.148064,0.143508,0.113895,0.769231,5.360195,1.0,0.092647,3.711465,0.954813,0.641026,0.730565,0.781441


In [599]:
price_map = (
    df_with_price[['TN','avg_price']]
    .drop_duplicates(subset='TN')
    .set_index('TN')['avg_price']
)


assoc_rules_3['consequent_item'] = assoc_rules_3['consequents'].apply(lambda s: next(iter(s)))
assoc_rules_3['consequent_TN']   = assoc_rules_3['consequent_item'].apply(lambda x: x.split('_',1)[0])


assoc_rules_3['avg_price_consequent'] = assoc_rules_3['consequent_TN'].map(price_map)

# Считаем score
assoc_rules_3['score'] = (
    assoc_rules_3['confidence']
  * assoc_rules_3['lift']
  * assoc_rules_3['avg_price_consequent']
)

# Сортируем
assoc_rules_3 = assoc_rules_3.sort_values('score', ascending=False)
assoc_rules_3[[
    'antecedents','consequents',
    'confidence','lift',
    'avg_price_consequent','score'
]].head(10)

,antecedents,consequents,confidence,lift,avg_price_consequent,score
0,(Аксессуары_Аксессуары ТВ-Аудио),(ТВ-Аудио_Телевизоры LCD),0.847826,7.919056,17472.296758,117308.696849
2,(Аксессуары_Картриджи и бумага),(Цифровая Техника_Периферийные устройства),0.964286,7.055357,11276.214820,76716.375530
4,(Аксессуары_Аксессуары для смартфонов и планше...,(Цифровая Техника_Мобильные средства связи),0.769231,5.360195,11276.214820,46494.395659
8,(Аксессуары_Аксессуары для бытовой техники),(Крупная бытовая техника_Стиральные машины),0.525000,4.348585,18947.654681,43257.629699
11,(Аксессуары_Аксессуары для комп./ноутбуков/при...,(Цифровая Техника_Периферийные устройства),0.522727,3.824621,11276.214820,22543.789977
6,(Аксессуары_Аксессуары для смартфонов и планше...,(Цифровая Техника_Планшеты),0.215385,4.976518,11276.214820,12086.585212
1,(ТВ-Аудио_Телевизоры LCD),(Аксессуары_Аксессуары ТВ-Аудио),0.829787,7.919056,1254.350720,8242.504069
5,(Цифровая Техника_Мобильные средства связи),(Аксессуары_Аксессуары для смартфонов и планше...,0.793651,5.360195,1254.350720,5336.162627
7,(Цифровая Техника_Планшеты),(Аксессуары_Аксессуары для смартфонов и планше...,0.736842,4.976518,1254.350720,4599.588893
12,(Аксессуары_Аксессуары для бытовой техники),(Мелкая бытовая техника_Уход за домом),0.300000,3.465789,4329.630674,4501.676525


In [600]:
assoc_rules_3['antecedent_items'] = assoc_rules_3['antecedents'].apply(lambda s: list(s))
assoc_rules_3['consequent_item']  = assoc_rules_3['consequents'].apply(lambda s: next(iter(s)))

#Собираем какие товары уже купил каждый клиент
customer_items = df_with_price.groupby('Phone_new')['TN_TK'].apply(set)

recs = []
for _, rule in assoc_rules_3.iterrows():
    A = set(rule['antecedent_items'])
    C = rule['consequent_item']
    for phone, items in customer_items.items():
        if A.issubset(items) and C not in items:
            recs.append({
                'Phone_new':     phone,
                'antecedents':   ", ".join(rule['antecedent_items']),
                'recommend_consequents':     C,
                'confidence':    rule['confidence'],
                'lift':          rule['lift'],
                'avg_price':     rule['avg_price_consequent'],
                'score':         rule['score']
            })

recommend_df_3 = pd.DataFrame(recs).drop_duplicates()

recommend_df_3.head()

,Phone_new,antecedents,recommend_consequents,confidence,lift,avg_price,score
0,55574848-48555649515078,Аксессуары_Аксессуары ТВ-Аудио,ТВ-Аудио_Телевизоры LCD,0.847826,7.919056,17472.296758,117308.696849
1,55574850-535149515556785,Аксессуары_Аксессуары ТВ-Аудио,ТВ-Аудио_Телевизоры LCD,0.847826,7.919056,17472.296758,117308.696849
2,55574851-49505650565272,Аксессуары_Аксессуары ТВ-Аудио,ТВ-Аудио_Телевизоры LCD,0.847826,7.919056,17472.296758,117308.696849
3,55574851-49505651565279,Аксессуары_Аксессуары ТВ-Аудио,ТВ-Аудио_Телевизоры LCD,0.847826,7.919056,17472.296758,117308.696849
4,55574851-50535454565673,Аксессуары_Аксессуары ТВ-Аудио,ТВ-Аудио_Телевизоры LCD,0.847826,7.919056,17472.296758,117308.696849


**Анализ MBA для кластера 4**

In [601]:
df = pd.read_csv('Data_for_MBA_all_clusters_true.csv')
df.set_index('id', inplace = True)
df.drop(['Unnamed: 0'], axis = 1, inplace = True)

In [602]:
df_4 = df.loc[df['Cluster'] == 4]

In [603]:
MarkBas_4 = df_4.drop(columns='Cluster', axis=1)

In [604]:
#Create function to hot encode the values
def encode_values(x):
    if x <= 0:
        return 0
    if x >= 1:
        return 1

#Apply function to data
basket_encoded_4 = MarkBas_4.applymap(encode_values)

/var/folders/m_/q7bdmmy93879kj5hcdgd5gsc0000gq/T/ipykernel_48164/3279359958.py:9: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  basket_encoded_4 = MarkBas_4.applymap(encode_values)


In [605]:
#filter for only invoices with 2 or more items
basket_filtered_4 = basket_encoded_4[(basket_encoded_4 > 0).sum(axis=1) >= 2]

In [606]:
basket_filtered_4.shape

(642, 95)

In [607]:
basket_filtered_4.to_csv('mba_5.csv')

In [608]:
#Generate the frequent itemsets
frequent_itemsets_4 = apriori(basket_filtered_4, min_support=0.03, use_colnames=True).sort_values("support",ascending=False)
frequent_itemsets_4.head()

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/mlxtend/frequent_patterns/fpcommon.py:161: DeprecationWarning: DataFrames with non-bool types result in worse computationalperformance and their support might be discontinued in the future.Please use a DataFrame with bool type
  warnings.warn(


,support,itemsets
8,0.537383,(Крупная бытовая техника_Холодильное оборудова...
5,0.422118,(Крупная бытовая техника_Стиральные машины)
4,0.291277,(Крупная бытовая техника_Встраиваемая техника)
22,0.266355,(Крупная бытовая техника_Холодильное оборудова...
7,0.247664,(Крупная бытовая техника_Плиты)


In [609]:
#Apply association rules
assoc_rules_4 = association_rules(frequent_itemsets_4, metric="lift", min_threshold=0.2).sort_values("lift",ascending=False).reset_index(drop=True)
assoc_rules_4.head(6)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,representativity,leverage,conviction,zhangs_metric,jaccard,certainty,kulczynski
0,(ТВ-Аудио_Телевизоры LCD),(Мелкая бытовая техника_Уход за домом),0.155763,0.118380,0.034268,0.220000,1.858421,1.0,0.015829,1.130282,0.547132,0.142857,0.115265,0.254737
1,(Мелкая бытовая техника_Уход за домом),(ТВ-Аудио_Телевизоры LCD),0.118380,0.155763,0.034268,0.289474,1.858421,1.0,0.015829,1.188185,0.523932,0.142857,0.158380,0.254737
2,(Крупная бытовая техника_Холодильное оборудова...,(Крупная бытовая техника_Стиральные машины),0.537383,0.422118,0.266355,0.495652,1.174202,1.0,0.039516,1.145800,0.320692,0.384270,0.127247,0.563324
3,(Крупная бытовая техника_Стиральные машины),(Крупная бытовая техника_Холодильное оборудова...,0.422118,0.537383,0.266355,0.630996,1.174202,1.0,0.039516,1.253692,0.256727,0.384270,0.202356,0.563324
4,(Мелкая бытовая техника_Приготовление пищи),(ТВ-Аудио_Телевизоры LCD),0.182243,0.155763,0.031153,0.170940,1.097436,1.0,0.002766,1.018306,0.108571,0.101523,0.017977,0.185470
5,(ТВ-Аудио_Телевизоры LCD),(Мелкая бытовая техника_Приготовление пищи),0.155763,0.182243,0.031153,0.200000,1.097436,1.0,0.002766,1.022196,0.105166,0.101523,0.021714,0.185470


In [610]:
price_map = (
    df_with_price[['TN','avg_price']]
    .drop_duplicates(subset='TN')
    .set_index('TN')['avg_price']
)


assoc_rules_4['consequent_item'] = assoc_rules_4['consequents'].apply(lambda s: next(iter(s)))
assoc_rules_4['consequent_TN']   = assoc_rules_4['consequent_item'].apply(lambda x: x.split('_',1)[0])


assoc_rules_4['avg_price_consequent'] = assoc_rules_4['consequent_TN'].map(price_map)

# Считаем score
assoc_rules_4['score'] = (
    assoc_rules_4['confidence']
  * assoc_rules_4['lift']
  * assoc_rules_4['avg_price_consequent']
)

# Сортируем
assoc_rules_4 = assoc_rules_4.sort_values('score', ascending=False)
assoc_rules_4[[
    'antecedents','consequents',
    'confidence','lift',
    'avg_price_consequent','score'
]].head(10)

,antecedents,consequents,confidence,lift,avg_price_consequent,score
3,(Крупная бытовая техника_Стиральные машины),(Крупная бытовая техника_Холодильное оборудова...,0.630996,1.174202,18947.654681,14038.639865
2,(Крупная бытовая техника_Холодильное оборудова...,(Крупная бытовая техника_Стиральные машины),0.495652,1.174202,18947.654681,11027.453343
1,(Мелкая бытовая техника_Уход за домом),(ТВ-Аудио_Телевизоры LCD),0.289474,1.858421,17472.296758,9399.466460
9,(Мелкая бытовая техника_СВЧ),(Крупная бытовая техника_Холодильное оборудова...,0.488372,0.908797,18947.654681,8409.556107
11,(Крупная бытовая техника_Плиты),(Крупная бытовая техника_Холодильное оборудова...,0.484277,0.901176,18947.654681,8269.106700
13,"(Крупная бытовая техника_Встраиваемая техника,...",(Крупная бытовая техника_Стиральные машины),0.373134,0.883957,18947.654681,6249.591203
15,(Мелкая бытовая техника_Уход за домом),(Крупная бытовая техника_Стиральные машины),0.368421,0.872791,18947.654681,6092.703958
20,"(Крупная бытовая техника_Встраиваемая техника,...",(Крупная бытовая техника_Холодильное оборудова...,0.390625,0.726902,18947.654681,5380.113819
16,(Крупная бытовая техника_Встраиваемая техника),(Крупная бытовая техника_Стиральные машины),0.342246,0.810782,18947.654681,5257.725802
6,(Мелкая бытовая техника_СВЧ),(Крупная бытовая техника_Встраиваемая техника),0.267442,0.918169,18947.654681,4652.727871


In [611]:
assoc_rules_4['antecedent_items'] = assoc_rules_4['antecedents'].apply(lambda s: list(s))
assoc_rules_4['consequent_item']  = assoc_rules_4['consequents'].apply(lambda s: next(iter(s)))

#Собираем какие товары уже купил каждый клиент
customer_items = df_with_price.groupby('Phone_new')['TN_TK'].apply(set)

recs = []
for _, rule in assoc_rules_4.iterrows():
    A = set(rule['antecedent_items'])
    C = rule['consequent_item']
    for phone, items in customer_items.items():
        if A.issubset(items) and C not in items:
            recs.append({
                'Phone_new':     phone,
                'antecedents':   ", ".join(rule['antecedent_items']),
                'recommend_consequents':     C,
                'confidence':    rule['confidence'],
                'lift':          rule['lift'],
                'avg_price':     rule['avg_price_consequent'],
                'score':         rule['score']
            })

recommend_df_4 = pd.DataFrame(recs).drop_duplicates()

recommend_df_4.head()

,Phone_new,antecedents,recommend_consequents,confidence,lift,avg_price,score
0,55485656-57565656575275,Крупная бытовая техника_Стиральные машины,Крупная бытовая техника_Холодильное оборудование,0.630996,1.174202,18947.654681,14038.639865
1,55574850-515150575554725,Крупная бытовая техника_Стиральные машины,Крупная бытовая техника_Холодильное оборудование,0.630996,1.174202,18947.654681,14038.639865
2,55574850-515656575650755,Крупная бытовая техника_Стиральные машины,Крупная бытовая техника_Холодильное оборудование,0.630996,1.174202,18947.654681,14038.639865
3,55574850-53495251565070,Крупная бытовая техника_Стиральные машины,Крупная бытовая техника_Холодильное оборудование,0.630996,1.174202,18947.654681,14038.639865
4,55574850-53545357545771,Крупная бытовая техника_Стиральные машины,Крупная бытовая техника_Холодильное оборудование,0.630996,1.174202,18947.654681,14038.639865


**Анализ MBA для кластера 5**

In [612]:
df = pd.read_csv('Data_for_MBA_all_clusters_true.csv')
df.set_index('id', inplace = True)
df.drop(['Unnamed: 0'], axis = 1, inplace = True)

In [613]:
df_5 = df.loc[df['Cluster'] == 5]

In [614]:
MarkBas_5 = df_5.drop(columns='Cluster', axis=1)

In [615]:
#Create function to hot encode the values
def encode_values(x):
    if x <= 0:
        return 0
    if x >= 1:
        return 1

#Apply function to data
basket_encoded_5 = MarkBas_5.applymap(encode_values)

/var/folders/m_/q7bdmmy93879kj5hcdgd5gsc0000gq/T/ipykernel_48164/765676802.py:9: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  basket_encoded_5 = MarkBas_5.applymap(encode_values)


In [616]:
#filter for only invoices with 2 or more items
basket_filtered_5 = basket_encoded_5[(basket_encoded_5 > 0).sum(axis=1) >= 2]

In [617]:
basket_filtered_5.shape

(461, 95)

In [618]:
basket_filtered_5.to_csv('mba_6.csv')

In [619]:
#Generate the frequent itemsets
frequent_itemsets_5 = apriori(basket_filtered_5, min_support=0.03, use_colnames=True).sort_values("support",ascending=False)
frequent_itemsets_5.head()

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/mlxtend/frequent_patterns/fpcommon.py:161: DeprecationWarning: DataFrames with non-bool types result in worse computationalperformance and their support might be discontinued in the future.Please use a DataFrame with bool type
  warnings.warn(


,support,itemsets
0,0.505423,(Мелкая бытовая техника_Приготовление пищи)
1,0.305857,(Мелкая бытовая техника_Красота и здоровье)
4,0.262473,(Мелкая бытовая техника_Уход за домом)
10,0.203905,"(Мелкая бытовая техника_Приготовление пищи, Ме..."
3,0.190889,(Спорт и активный отдых_Тяжелая атлетика)


In [620]:
#Apply association rules
assoc_rules_5 = association_rules(frequent_itemsets_5, metric="lift", min_threshold=0.2).sort_values("lift",ascending=False).reset_index(drop=True)
assoc_rules_5.head()

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,representativity,leverage,conviction,zhangs_metric,jaccard,certainty,kulczynski
0,(Спорт и активный отдых_Аксессуары для спортив...,(Спорт и активный отдых_Велосипеды и аксессуары),0.121475,0.130152,0.097614,0.803571,6.174107,1.0,0.081804,4.428318,0.953909,0.633803,0.774181,0.776786
1,(Спорт и активный отдых_Велосипеды и аксессуары),(Спорт и активный отдых_Аксессуары для спортив...,0.130152,0.121475,0.097614,0.750000,6.174107,1.0,0.081804,3.514100,0.963425,0.633803,0.715432,0.776786
2,(Спорт и активный отдых_Фитнес),(Спорт и активный отдых_Тяжелая атлетика),0.147505,0.190889,0.097614,0.661765,3.466745,1.0,0.069457,2.392153,0.834662,0.405405,0.581967,0.586564
3,(Спорт и активный отдых_Тяжелая атлетика),(Спорт и активный отдых_Фитнес),0.190889,0.147505,0.097614,0.511364,3.466745,1.0,0.069457,1.744640,0.879416,0.405405,0.426816,0.586564
4,(Спорт и активный отдых_Тренажеры),(Спорт и активный отдых_Тяжелая атлетика),0.114967,0.190889,0.058568,0.509434,2.668739,1.0,0.036622,1.649341,0.706518,0.236842,0.393697,0.408126


In [621]:
price_map = (
    df_with_price[['TN','avg_price']]
    .drop_duplicates(subset='TN')
    .set_index('TN')['avg_price']
)


assoc_rules_5['consequent_item'] = assoc_rules_5['consequents'].apply(lambda s: next(iter(s)))
assoc_rules_5['consequent_TN']   = assoc_rules_5['consequent_item'].apply(lambda x: x.split('_',1)[0])


assoc_rules_5['avg_price_consequent'] = assoc_rules_5['consequent_TN'].map(price_map)

# Считаем score
assoc_rules_5['score'] = (
    assoc_rules_5['confidence']
  * assoc_rules_5['lift']
  * assoc_rules_5['avg_price_consequent']
)

# Сортируем
assoc_rules_5 = assoc_rules_5.sort_values('score', ascending=False)
assoc_rules_5[[
    'antecedents','consequents',
    'confidence','lift',
    'avg_price_consequent','score'
]].head(10)

,antecedents,consequents,confidence,lift,avg_price_consequent,score
0,(Спорт и активный отдых_Аксессуары для спортив...,(Спорт и активный отдых_Велосипеды и аксессуары),0.803571,6.174107,2910.316521,14439.058408
1,(Спорт и активный отдых_Велосипеды и аксессуары),(Спорт и активный отдых_Аксессуары для спортив...,0.750000,6.174107,2910.316521,13476.454514
2,(Спорт и активный отдых_Фитнес),(Спорт и активный отдых_Тяжелая атлетика),0.661765,3.466745,2910.316521,6676.758685
3,(Спорт и активный отдых_Тяжелая атлетика),(Спорт и активный отдых_Фитнес),0.511364,3.466745,2910.316521,5159.313529
6,(Мелкая бытовая техника_СВЧ),(Мелкая бытовая техника_Приготовление пищи),0.684932,1.355165,4329.630674,4018.742174
4,(Спорт и активный отдых_Тренажеры),(Спорт и активный отдых_Тяжелая атлетика),0.509434,2.668739,2910.316521,3956.710423
8,(Мелкая бытовая техника_Красота и здоровье),(Мелкая бытовая техника_Приготовление пищи),0.666667,1.319027,4329.630674,3807.267031
11,(Мелкая бытовая техника_Уход за домом),(Мелкая бытовая техника_Приготовление пищи),0.570248,1.128259,4329.630674,2785.629141
5,(Спорт и активный отдых_Тяжелая атлетика),(Спорт и активный отдых_Тренажеры),0.306818,2.668739,2910.316521,2383.018777
9,(Мелкая бытовая техника_Приготовление пищи),(Мелкая бытовая техника_Красота и здоровье),0.403433,1.319027,4329.630674,2303.968461


In [622]:
assoc_rules_5['antecedent_items'] = assoc_rules_5['antecedents'].apply(lambda s: list(s))
assoc_rules_5['consequent_item']  = assoc_rules_5['consequents'].apply(lambda s: next(iter(s)))

#Собираем какие товары уже купил каждый клиент
customer_items = df_with_price.groupby('Phone_new')['TN_TK'].apply(set)

recs = []
for _, rule in assoc_rules_5.iterrows():
    A = set(rule['antecedent_items'])
    C = rule['consequent_item']
    for phone, items in customer_items.items():
        if A.issubset(items) and C not in items:
            recs.append({
                'Phone_new':     phone,
                'antecedents':   ", ".join(rule['antecedent_items']),
                'recommend_consequents':     C,
                'confidence':    rule['confidence'],
                'lift':          rule['lift'],
                'avg_price':     rule['avg_price_consequent'],
                'score':         rule['score']
            })

recommend_df_5 = pd.DataFrame(recs).drop_duplicates()

recommend_df_5.head()

,Phone_new,antecedents,recommend_consequents,confidence,lift,avg_price,score
0,55574848-545751524853795,Спорт и активный отдых_Аксессуары для спортивн...,Спорт и активный отдых_Велосипеды и аксессуары,0.803571,6.174107,2910.316521,14439.058408
1,55574850-535555495650795,Спорт и активный отдых_Аксессуары для спортивн...,Спорт и активный отдых_Велосипеды и аксессуары,0.803571,6.174107,2910.316521,14439.058408
2,55574851-48535549545076,Спорт и активный отдых_Аксессуары для спортивн...,Спорт и активный отдых_Велосипеды и аксессуары,0.803571,6.174107,2910.316521,14439.058408
3,55574851-49544850495678,Спорт и активный отдых_Аксессуары для спортивн...,Спорт и активный отдых_Велосипеды и аксессуары,0.803571,6.174107,2910.316521,14439.058408
4,55574851-495749505052735,Спорт и активный отдых_Аксессуары для спортивн...,Спорт и активный отдых_Велосипеды и аксессуары,0.803571,6.174107,2910.316521,14439.058408


**Анализ MBA для кластера 6**

In [623]:
df = pd.read_csv('Data_for_MBA_all_clusters_true.csv')
df.set_index('id', inplace = True)
df.drop(['Unnamed: 0'], axis = 1, inplace = True)

In [624]:
df_6 = df.loc[df['Cluster'] == 6]

In [625]:
MarkBas_6 = df_6.drop(columns='Cluster', axis=1)

In [626]:
#Create function to hot encode the values
def encode_values(x):
    if x <= 0:
        return 0
    if x >= 1:
        return 1

#Apply function to data
basket_encoded_6 = MarkBas_6.applymap(encode_values)

/var/folders/m_/q7bdmmy93879kj5hcdgd5gsc0000gq/T/ipykernel_48164/3783226430.py:9: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  basket_encoded_6 = MarkBas_6.applymap(encode_values)


In [627]:
#filter for only invoices with 2 or more items
basket_filtered_6 = basket_encoded_6[(basket_encoded_6 > 0).sum(axis=1) >= 2]

In [628]:
basket_filtered_6.shape

(322, 95)

In [629]:
basket_filtered_6.to_csv('mba_7.csv')

In [630]:
#Generate the frequent itemsets
frequent_itemsets_6 = apriori(basket_filtered_6, min_support=0.03, use_colnames=True).sort_values("support",ascending=False)
frequent_itemsets_6.head()

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/mlxtend/frequent_patterns/fpcommon.py:161: DeprecationWarning: DataFrames with non-bool types result in worse computationalperformance and their support might be discontinued in the future.Please use a DataFrame with bool type
  warnings.warn(


,support,itemsets
18,0.232919,(Детские товары_Прогулки и путешествие)
9,0.189441,(Товары для дома_Хозтовары)
13,0.183230,(Детские товары_Игрушки)
0,0.142857,(Мелкая бытовая техника_Приготовление пищи)
5,0.139752,(Детские товары_Детская комната)


In [631]:
#Apply association rules
assoc_rules_6 = association_rules(frequent_itemsets_6, metric="lift", min_threshold=0.2).sort_values("lift",ascending=False).reset_index(drop=True)
assoc_rules_6.head()

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,representativity,leverage,conviction,zhangs_metric,jaccard,certainty,kulczynski
0,(Крупная бытовая техника_Посудомоечные машины),(Товары для дома_Хозтовары),0.043478,0.189441,0.040373,0.928571,4.901639,1.0,0.032136,11.347826,0.832168,0.209677,0.911877,0.570843
1,(Товары для дома_Хозтовары),(Крупная бытовая техника_Посудомоечные машины),0.189441,0.043478,0.040373,0.213115,4.901639,1.0,0.032136,1.215580,0.982022,0.209677,0.177347,0.570843
2,(Крупная бытовая техника_Встраиваемая техника),(Сделай сам_Сантехника),0.130435,0.093168,0.049689,0.380952,4.088889,1.0,0.037537,1.464883,0.868750,0.285714,0.317352,0.457143
3,(Сделай сам_Сантехника),(Крупная бытовая техника_Встраиваемая техника),0.093168,0.130435,0.049689,0.533333,4.088889,1.0,0.037537,1.863354,0.833048,0.285714,0.463333,0.457143
4,(Детские товары_Прогулки и путешествие),(Детские товары_Аксессуары детство),0.232919,0.077640,0.049689,0.213333,2.747733,1.0,0.031606,1.172492,0.829200,0.190476,0.147116,0.426667


In [632]:
price_map = (
    df_with_price[['TN','avg_price']]
    .drop_duplicates(subset='TN')
    .set_index('TN')['avg_price']
)


assoc_rules_6['consequent_item'] = assoc_rules_6['consequents'].apply(lambda s: next(iter(s)))
assoc_rules_6['consequent_TN']   = assoc_rules_6['consequent_item'].apply(lambda x: x.split('_',1)[0])


assoc_rules_6['avg_price_consequent'] = assoc_rules_6['consequent_TN'].map(price_map)

# Считаем score
assoc_rules_6['score'] = (
    assoc_rules_6['confidence']
  * assoc_rules_6['lift']
  * assoc_rules_6['avg_price_consequent']
)

# Сортируем
assoc_rules_6 = assoc_rules_6.sort_values('score', ascending=False)
assoc_rules_6[[
    'antecedents','consequents',
    'confidence','lift',
    'avg_price_consequent','score'
]].head(10)

,antecedents,consequents,confidence,lift,avg_price_consequent,score
3,(Сделай сам_Сантехника),(Крупная бытовая техника_Встраиваемая техника),0.533333,4.088889,18947.654681,41319.922503
1,(Товары для дома_Хозтовары),(Крупная бытовая техника_Посудомоечные машины),0.213115,4.901639,18947.654681,19792.941076
6,(Товары для дома_Хозтовары),(Крупная бытовая техника_Холодильное оборудова...,0.163934,2.111475,18947.654681,6558.607694
0,(Крупная бытовая техника_Посудомоечные машины),(Товары для дома_Хозтовары),0.928571,4.901639,1283.735692,5842.951563
5,(Детские товары_Аксессуары детство),(Детские товары_Прогулки и путешествие),0.640000,2.747733,3137.673723,5517.754033
2,(Крупная бытовая техника_Встраиваемая техника),(Сделай сам_Сантехника),0.380952,4.088889,3328.979810,5185.458497
9,(Товары для дома_Хозтовары),(Крупная бытовая техника_Встраиваемая техника),0.180328,1.382514,18947.654681,4723.759113
4,(Детские товары_Прогулки и путешествие),(Детские товары_Аксессуары детство),0.213333,2.747733,3137.673723,1839.251344
7,(Крупная бытовая техника_Холодильное оборудова...,(Товары для дома_Хозтовары),0.400000,2.111475,1283.735692,1084.230539
13,(Детские товары_Игрушки),(Детские товары_Прогулки и путешествие),0.254237,1.091525,3137.673723,870.724739


In [633]:
assoc_rules_6['antecedent_items'] = assoc_rules_6['antecedents'].apply(lambda s: list(s))
assoc_rules_6['consequent_item']  = assoc_rules_6['consequents'].apply(lambda s: next(iter(s)))

#Собираем какие товары уже купил каждый клиент
customer_items = df_with_price.groupby('Phone_new')['TN_TK'].apply(set)

recs = []
for _, rule in assoc_rules_6.iterrows():
    A = set(rule['antecedent_items'])
    C = rule['consequent_item']
    for phone, items in customer_items.items():
        if A.issubset(items) and C not in items:
            recs.append({
                'Phone_new':     phone,
                'antecedents':   ", ".join(rule['antecedent_items']),
                'recommend_consequents':     C,
                'confidence':    rule['confidence'],
                'lift':          rule['lift'],
                'avg_price':     rule['avg_price_consequent'],
                'score':         rule['score']
            })

recommend_df_6 = pd.DataFrame(recs).drop_duplicates()

recommend_df_6.head()

,Phone_new,antecedents,recommend_consequents,confidence,lift,avg_price,score
0,55574848-53565450534870,Сделай сам_Сантехника,Крупная бытовая техника_Встраиваемая техника,0.533333,4.088889,18947.654681,41319.922503
1,55574850-49574848575275,Сделай сам_Сантехника,Крупная бытовая техника_Встраиваемая техника,0.533333,4.088889,18947.654681,41319.922503
2,55574850-505454555052765,Сделай сам_Сантехника,Крупная бытовая техника_Встраиваемая техника,0.533333,4.088889,18947.654681,41319.922503
3,55574850-505454575553715,Сделай сам_Сантехника,Крупная бытовая техника_Встраиваемая техника,0.533333,4.088889,18947.654681,41319.922503
4,55574850-53495052544870,Сделай сам_Сантехника,Крупная бытовая техника_Встраиваемая техника,0.533333,4.088889,18947.654681,41319.922503
